# Sentence Transformers 训练概述

官方文档：https://sbert.net/docs/sentence_transformer/training_overview.html

## 训练六要素

1. **Model** — 基础模型
2. **Dataset** — 训练数据
3. **Loss** — 损失函数
4. **TrainingArguments** — 训练参数
5. **Evaluator** — 评估器（可选）
6. **Trainer** — 训练器

## 数据格式

训练数据格式决定了可以使用哪些损失函数：

| 数据格式 | 示例 | 适用 Loss |
|---------|------|----------|
| **正样本对** (anchor, positive) | (query, relevant_doc) | MultipleNegativesRankingLoss |
| **三元组** (anchor, positive, negative) | (query, good_doc, bad_doc) | TripletLoss, MNRL |
| **带分数的句子对** (sent1, sent2, score) | ("天气好", "阳光明媚", 0.9) | CosineSimilarityLoss, CoSENTLoss |
| **带类别的文本** (text, label) | ("好评", 1) | BatchAllTripletLoss |

## 损失函数选择指南

最常用的两种场景：

### 1. 有正样本对/三元组 → MultipleNegativesRankingLoss (MNRL)
- 训练效果最好的 embedding 模型大多使用此 loss
- 利用 in-batch negatives，batch 越大效果越好
- CachedMultipleNegativesRankingLoss 可在显存有限时模拟大 batch

### 2. 有相似度分数 → CoSENTLoss
- 比传统的 CosineSimilarityLoss 效果更好
- 适合 STS（语义文本相似度）任务

### Loss 修饰器（可叠加）
- **MatryoshkaLoss** — 支持截断向量维度（如 768→256→128）
- **AdaptiveLayerLoss** — 支持使用更少的 Transformer 层

## 数据集示例

In [ ]:
from datasets import Dataset, load_dataset

# 格式1：带分数的句子对（STS）
sts_dataset = load_dataset("sentence-transformers/stsb", split="train[:5]")
print("=== STS 数据格式 ===")
print(sts_dataset)
print(sts_dataset[0])

In [ ]:
# 格式2：三元组（NLI）
nli_dataset = load_dataset("sentence-transformers/all-nli", "triplet", split="train[:5]")
print("=== NLI 三元组格式 ===")
print(nli_dataset)
print(nli_dataset[0])

In [ ]:
# 格式3：自定义数据集
custom_dataset = Dataset.from_dict({
    "sentence1": ["深度学习很有趣", "Python是好语言"],
    "sentence2": ["神经网络很有意思", "Java也不错"],
    "label": [0.9, 0.5],
})
print("=== 自定义数据格式 ===")
print(custom_dataset[0])